In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn.functional as F

import sys
sys.path.append('../dgmr_original')
import dgmr

In [4]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [9]:
FORECAST = 24
# modify DGMR to handle 128/8 input
ccs = dgmr.common.ContextConditioningStack(
    input_channels=2,
    conv_type='standard',
    output_channels=192,
)

lcs = dgmr.common.LatentConditioningStack(
    shape=(8 * 1, 128 // 32, 128 // 32),
    output_channels=384,
)

sampler = dgmr.generators.Sampler(
    forecast_steps=FORECAST,
    latent_channels=384,
    context_channels=192,
)

model = dgmr.generators.Generator(ccs, lcs, sampler)
model = model.to(DEVICE)

In [10]:
x = torch.rand((16, 4, 2, 128, 128)).to(DEVICE)
out = model(x)

RuntimeError: Given groups=1, weight of size [192, 576, 3, 3], expected input[16, 768, 4, 4] to have 576 channels, but got 768 channels instead

In [ ]:
out.shape

In [ ]:
y = torch.rand((16, FORECAST, 1, 128, 128)).to(DEVICE)
loss = F.mse_loss(y, out)

In [ ]:
loss

In [ ]:
loss.shape

In [ ]:
loss.backward()